In [71]:
import requests
import json 
import random 

In [72]:
presidential_national_vote_url = "https://www.politico.com/election-data/results__2024-11-05__collections__2024-11-05-collection-president__summaries/data.json"
presidential_county_base_url_prefix = "https://www.politico.com/election-data/results__2024-11-05__contests__ap:" 
presidential_national_vote_url_suffix = ":0__counties/data.json"

In [73]:
with open("us_states.json", "r") as infile:
    national_feature_map = json.load(infile)
with open("candidate_ids.json", "r") as infile:
    candidates = json.load(infile)
with open("county_feature_list.json", "r") as infile:
    county_feature_map = json.load(infile)



In [74]:
v_request = requests.get(presidential_national_vote_url)
v_cont = v_request.content
v_dict = json.loads(v_cont)

In [75]:
v_dict

{'eventId': '2024-11-05',
 'id': '2024-11-05-collection-president',
 'contests': [{'eventId': '2024-11-05',
   'resultsType': 'live',
   'id': 'ap:02:0',
   'timestamp': '2024-11-01T18:30:30.939000Z',
   'results': [{'votes': 0, 'id': '167861'},
    {'votes': 0, 'id': '61513'},
    {'votes': 0, 'id': '64984'},
    {'votes': 0, 'id': '69459'},
    {'votes': 0, 'id': '71841'},
    {'votes': 0, 'id': '71845'},
    {'votes': 0, 'id': '8639'},
    {'votes': 0, 'id': '895'}],
   'progress': {'mode': 'eevp', 'pct': 0},
   'status': {'tabulation': 'before', 'recount': 'none'},
   'calls': [],
   'evs': 3},
  {'eventId': '2024-11-05',
   'resultsType': 'live',
   'id': 'ap:01:0',
   'timestamp': '2024-11-01T20:50:14.010000Z',
   'results': [{'votes': 0, 'id': '64984'},
    {'votes': 0, 'id': '69459'},
    {'votes': 0, 'id': '71841'},
    {'votes': 0, 'id': '8639'},
    {'votes': 0, 'id': '895'}],
   'progress': {'mode': 'eevp', 'pct': 0},
   'status': {'tabulation': 'before', 'recount': 'none'}

In [76]:

for contest in v_dict["contests"]:
    tmp_id = contest["id"][3:5]
    harris = 0 
    trump = 0 
    third_party = 0 
    progress = contest['progress']['pct']
    for result in contest['results']:
        if candidates[result['id']] == 'Donald Trump': 
            trump += result['votes'] 
        elif candidates[result['id']] == 'Kamala Harris': 
            harris += result['votes']
        else: 
            third_party += result['votes']
    if harris+trump+third_party != 0:
        harris = harris/(harris+trump+third_party)
        trump = trump/(harris+trump+third_party)
        third_party = third_party/(harris+trump+third_party)
    for i in national_feature_map['features']:
        if i['id'] == tmp_id:
            tmp_property_dict = i['properties']
            tmp_property_dict["harris"] = harris
            tmp_property_dict["trump"] = trump
            tmp_property_dict["third_party"] = third_party
            tmp_property_dict["progress"] = progress
            tmp_property_dict["margin"] = harris - trump
            tmp_property_dict["margin"] = random.randint(-15,15)
            i["properties"] = tmp_property_dict
with open("us_states.js", "w") as outfile:
    outfile.write(f"var statesData = {json.dumps(national_feature_map)}")

In [85]:
state_json

{'eventId': '2024-11-05',
 'id': 'ap:01:0',
 'resultsType': 'live',
 'timestamp': '2024-11-01T20:50:14.010000Z',
 'counties': [{'id': '01001',
   'results': [{'id': '64984', 'votes': 0},
    {'id': '69459', 'votes': 0},
    {'id': '71841', 'votes': 0},
    {'id': '8639', 'votes': 0},
    {'id': '895', 'votes': 0}],
   'progress': {'mode': 'eevp', 'pct': 0}},
  {'id': '01003',
   'results': [{'id': '64984', 'votes': 0},
    {'id': '69459', 'votes': 0},
    {'id': '71841', 'votes': 0},
    {'id': '8639', 'votes': 0},
    {'id': '895', 'votes': 0}],
   'progress': {'mode': 'eevp', 'pct': 0}},
  {'id': '01005',
   'results': [{'id': '64984', 'votes': 0},
    {'id': '69459', 'votes': 0},
    {'id': '71841', 'votes': 0},
    {'id': '8639', 'votes': 0},
    {'id': '895', 'votes': 0}],
   'progress': {'mode': 'eevp', 'pct': 0}},
  {'id': '01007',
   'results': [{'id': '64984', 'votes': 0},
    {'id': '69459', 'votes': 0},
    {'id': '71841', 'votes': 0},
    {'id': '8639', 'votes': 0},
    {'i

In [88]:
for feature in national_feature_map['features']:
    state_request = requests.get(f"{presidential_county_base_url_prefix}{feature['id']}{presidential_national_vote_url_suffix}")
    state_content = state_request.content
    state_json = json.loads(state_content)
    for contest in state_json["counties"]:
        tmp_id = str(int(contest["id"]))
        print(tmp_id)
        harris = 0 
        trump = 0 
        third_party = 0 
        progress = contest['progress']['pct']
        for result in contest['results']:
            if candidates[result['id']] == 'Donald Trump': 
                trump += result['votes'] 
            elif candidates[result['id']] == 'Kamala Harris': 
                harris += result['votes']
            else: 
                third_party += result['votes']
        if harris+trump+third_party != 0:
            harris = harris/(harris+trump+third_party)
            trump = trump/(harris+trump+third_party)
            third_party = third_party/(harris+trump+third_party)
        for i in county_feature_map['features']:
            if i['id'] == tmp_id:
                tmp_property_dict = i['properties']
                tmp_property_dict["harris"] = harris
                tmp_property_dict["trump"] = trump
                tmp_property_dict["third_party"] = third_party
                tmp_property_dict["progress"] = progress
                tmp_property_dict["margin"] = harris - trump
                tmp_property_dict["margin"] = random.randint(-15,15)
                i["properties"] = tmp_property_dict

with open("us_counties.js", "w") as outfile:
    outfile.write(f"var countyData = {json.dumps(county_feature_map)}")

1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2000
0
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027
6029
6031
6033
6035
6037
6039
6041
6043
6045
6047
6049
6051
6053
6055
6057
6059
6061
6063
6065
6067
6069
6071
6073
6075
6077
6079
6081
608

In [89]:
county_feature_map

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '1001',
   'properties': {'name': 'Autauga',
    'harris': 0,
    'trump': 0,
    'third_party': 0,
    'progress': 0,
    'margin': 13},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-86.411786, 32.706342],
      [-86.411786, 32.410587],
      [-86.499417, 32.344863],
      [-86.817079, 32.339387],
      [-86.915664, 32.662526],
      [-86.411786, 32.706342]]]}},
  {'type': 'Feature',
   'id': '1003',
   'properties': {'name': 'Baldwin',
    'harris': 0,
    'trump': 0,
    'third_party': 0,
    'progress': 0,
    'margin': 5},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.76459, 31.298768],
      [-87.616713, 31.243998],
      [-87.600282, 30.997536],
      [-87.518128, 30.280057],
      [-88.005575, 30.685351],
      [-87.972714, 31.161844],
      [-87.945329, 31.194706],
      [-87.76459, 31.298768]]]}},
  {'type': 'Feature',
   'id': '1005',
   'properties': {'name': 'Barbour',
    '

In [77]:
tmp_r = requests.get("https://raw.githubusercontent.com/python-visualization/folium/master/tests/us-counties.json")
tmp_c = tmp_r.content
tmp_d = json.loads(tmp_c)

In [78]:
with open("county_feature_list.json", "w") as outfile:
     json.dump(tmp_d,outfile)

In [79]:
for feature in tmp_d["features"]:
    if feature["properties"]["name"] == "Coconino":
        print(feature)

{'type': 'Feature', 'id': '4005', 'properties': {'name': 'Coconino'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-110.751309, 37.00574], [-110.751309, 34.261793], [-111.556419, 34.469916], [-111.775497, 34.979272], [-112.334144, 34.973795], [-112.438206, 35.258596], [-113.336425, 35.526966], [-113.352855, 36.047275], [-112.629899, 36.392322], [-112.542268, 37.000263], [-111.414018, 37.000263], [-110.751309, 37.00574]]]}}


In [80]:
candidate_info = {}